# Project : Birds vs Drone Detection
## Category : Binary Classification

### Import necessary Libraries

In [ ]:
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
import glob
from PIL import Image as PILImage
import os
HOME = os.getcwd()

In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

### Dataset

In [ ]:
import shutil
import random

# Define the paths
dataset_dir = '/kaggle/input/birds-vs-drone-dataset/BirdVsDrone'
train_dir = 'train'
test_dir = 'test'
val_dir = 'val'
categories = ['Birds', 'Drones']

In [ ]:
for category in categories:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(test_dir, category), exist_ok=True)
    os.makedirs(os.path.join(val_dir, category), exist_ok=True)

In [ ]:
def split_data(source_dir, train_dir, test_dir, val_dir, split_ratio=(0.7, 0.15, 0.15)):
    files = os.listdir(source_dir)
    random.shuffle(files)
    
    train_split = int(split_ratio[0] * len(files))
    val_split = int(split_ratio[1] * len(files))
    
    train_files = files[:train_split]
    val_files = files[train_split:train_split + val_split]
    test_files = files[train_split + val_split:]
    
    for f in train_files:
        shutil.copy(os.path.join(source_dir, f), os.path.join(train_dir, f))
    for f in val_files:
        shutil.copy(os.path.join(source_dir, f), os.path.join(val_dir, f))
    for f in test_files:
        shutil.copy(os.path.join(source_dir, f), os.path.join(test_dir, f))

In [ ]:
for category in categories:
    source_dir = os.path.join(dataset_dir, category)
    split_data(source_dir, os.path.join(train_dir, category), os.path.join(test_dir, category), os.path.join(val_dir, category))

## Training Model

In [ ]:
!yolo task=classify mode=train model=yolov8m-cls.pt data='/kaggle/working/' epochs=10 imgsz=128

## Validation

In [ ]:
!yolo task=classify mode=val model=/kaggle/working/runs/classify/train2/weights/best.pt data='/kaggle/working/'

## Testing

### Drones Prediction

In [ ]:
!yolo task = classify mode = predict model=/kaggle/working/runs/classify/train2/weights/best.pt conf=0.25 source="/kaggle/working/test/Drones"

### Birds Pridiction

In [ ]:
!yolo task = classify mode = predict model=/kaggle/working/runs/classify/train2/weights/best.pt conf=0.25 source="/kaggle/working/test/Birds"

## Confusion Matrix

In [ ]:
Image(filename=f'/kaggle/working/runs/classify/val/confusion_matrix.png')

## Training Accuracy & Loss over Epochs

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Sample data: Replace these with your actual captured metrics
train_accuracy = [0.911, 0.919, 0.968, 0.968, 0.968, 0.944, 0.952, 0.96, 0.96, 0.96]
train_loss = [0.6233, 0.3251, 0.2051, 0.153, 0.1202, 0.1229, 0.09469, 0.08621, 0.07422, 0.09751]

# Create a DataFrame for easier plotting with Seaborn
epochs = range(1, 11)
data = {
    'Epoch': epochs,
    'Train Accuracy': train_accuracy,
    'Train Loss': train_loss
}
df = pd.DataFrame(data)

# Plot Accuracy
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.lineplot(x='Epoch', y='Train Accuracy', data=df, label='Train Accuracy', marker='o')
plt.title('Training Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot Loss
plt.subplot(1, 2, 2)
sns.lineplot(x='Epoch', y='Train Loss', data=df, label='Train Loss', marker='o')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig("acc-loss-plot.png")
plt.show()

In [ ]:
def resize_image(image_path, size=(128, 128)):
    image = PILImage.open(image_path)
    return image.resize(size, PILImage.ANTIALIAS)

# Paths to the images
predict2_paths = glob.glob('/kaggle/working/runs/classify/predict2/*')[:3]
predict3_paths = glob.glob('/kaggle/working/runs/classify/predict3/*')[:3]

# Ensure both sets have the same number of images
num_images = min(len(predict2_paths), len(predict3_paths))
fig, axes = plt.subplots(num_images, 2, figsize=(10, num_images * 5))
for i in range(num_images):
    # Resize images
    img2 = resize_image(predict2_paths[i])
    img3 = resize_image(predict3_paths[i])
    
    # Display Predict 2 image
    axes[i, 0].imshow(img2)
    axes[i, 0].axis('off')  # Hide the axes
    axes[i, 0].set_title(f'Drone Test - Image {i+1}')
    
    # Display Predict 3 image
    axes[i, 1].imshow(img3)
    axes[i, 1].axis('off')  # Hide the axes
    axes[i, 1].set_title(f'Bird Test - Image {i+1}')

# Adjust layout
plt.tight_layout()
plt.show()